In [2]:
import pandas as pd
data = pd.read_csv('output.csv')
data

,name,link
0,Harry Potter and the Deathly Hallows (Harry Po...,/book/show/136251.Harry_Potter_and_the_Deathly...
1,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games
2,The Kite Runner,/book/show/77203.The_Kite_Runner
3,The Book Thief,/book/show/19063.The_Book_Thief
4,Harry Potter and the Half-Blood Prince (Harry ...,/book/show/1.Harry_Potter_and_the_Half_Blood_P...
...,...,...
195,The Bookseller of Kabul,/book/show/9838.The_Bookseller_of_Kabul
196,If I Stay,/book/show/4374400-if-i-stay
197,Dear John,/book/show/5526.Dear_John
198,"Sex, Drugs, and Cocoa Puffs: A Low Culture Man...",/book/show/599.Sex_Drugs_and_Cocoa_Puffs


In [2]:
for i in range(len(data['link'])):
    link = "https://www.goodreads.com" + data['link'][i]
    data['link'][i] = link
#data[data['link'] == '/book/show/1.Harry_Potter_and_the_Half_Blood_Prince']
(data[data['link'] == 'https://www.goodreads.com/book/show/1.Harry_Potter_and_the_Half_Blood_Prince'])

,name,link
4,Harry Potter and the Half-Blood Prince (Harry ...,https://www.goodreads.com/book/show/1.Harry_Po...


In [ ]:
from scrapy.crawler import CrawlerProcess
from scrapy import Spider
class MySpider(Spider):
    name = 'book'
    allowed_domains = ['https://www.goodreads.com']
    start_urls = list(data['link'])[0:10]

    def parse(self, response):
        table = response.xpath('/html/body/div[2]/div[3]')
        link = response.request.url
        yield{
        'title' : (data[data['link'] == link])['name'].values[0],  
        'author': table.xpath('//*[@id="bookAuthors"]/span[2]/div/a/span//text()').extract(),
        'book_link': response.request.url,
        'genre_and_vote' : table.xpath('div[1]/div[2]/div[3]/div[6]/div/div[2]/div/div[1]/div[1]/a//text()').extract()
        }

process = CrawlerProcess(settings={
    "FEEDS": {
        "test.csv": {"format": "csv"},
    },
})
process.crawl(MySpider)
process.start()

In [20]:
import pandas as pd
review_count = []
pages = []
publishers = []
result = pd.read_csv('test.csv')
for i in range(len(result['rating_count'])):
    if not result['author'].isna()[i]:
        review_count.append(result['rating_count'][i].split()[4])
        pages.append(str(result['num_of_page'][i]).split()[0])

        result['rating_count'][i] = result['rating_count'][i].split()[1]

        date_and_publisher = result['date_published'][i].split('by')
        date = date_and_publisher[0].split()
        date.remove('Published')
        result['date_published'][i] = " ".join(date)
        try:            
            publisher = date_and_publisher[1].strip()
            publishers.append(publisher)
        except:
            publishers.append('NaN')


    else:
        review_count.append('NaN')
        pages.append('NaN')
        publishers.append('NaN')

result.insert(8,column="review_count",value=review_count)
result['num_of_page'] = pd.DataFrame(pages)
result.insert(6,'publisher',publishers,True)


C:\Users\phucl\AppData\Local\Temp/ipykernel_4448/2224460629.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['rating_count'][i] = result['rating_count'][i].split()[1]
C:\Users\phucl\AppData\Local\Temp/ipykernel_4448/2224460629.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['date_published'][i] = " ".join(date)


In [17]:
import pandas as pd
result = pd.read_csv('test.csv')
a = result['date_published'][83].split('by')
a[0].split()

['Published', ',']